If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers seqeval sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB

In [ ]:
import transformers

print(transformers.__version__)

4.28.1


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Fine-tuning a model on a token classification task

In [ ]:
task = "ner"
model_checkpoint = "emilyalsentzer/Bio_ClinicalBERT"
batch_size = 16

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
# # If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import DatasetDict

dataset_split = load_dataset('/content/drive/MyDrive/CLEF2023/custom_dataset', split=[
    'train[0%:100%]', 'validation[0%:100%]'
])

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

Generating train split: 0 examples [00:00, ? examples/s]

Dataset custom_dataset downloaded and prepared to /root/.cache/huggingface/datasets/custom_dataset/customconll2003/1.0.0/ed0f243792f3823216022d17200b67aedfefa9c87ce1cf26ac4579b06575386d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 10709
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1190
    })
})

In [ ]:
datasets["train"][0]

{'id': '0',
 'tokens': ['Paciente',
  'varón',
  'de',
  '35',
  'años',
  'con',
  'tumoración',
  'en',
  'polo',
  'superior',
  'de',
  'teste',
  'derecho',
  'hallada',
  'de',
  'manera',
  'casual',
  'durante',
  'una',
  'autoexploración',
  ',',
  'motivo',
  'por',
  'el',
  'cual',
  'acude',
  'a',
  'consulta',
  'de',
  'urología',
  'donde',
  'se',
  'realiza',
  'exploración',
  'física',
  ',',
  'apreciando',
  'masa',
  'de',
  '1cm',
  'aproximado',
  'de',
  'diámetro',
  'dependiente',
  'de',
  'epidídimo',
  ',',
  'y',
  'ecografía',
  'testicular',
  ',',
  'que',
  'se',
  'informa',
  'como',
  'lesión',
  'nodular',
  'sólida',
  'en',
  'cabeza',
  'de',
  'epidídimo',
  'derecho',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
 

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PROCEDIMIENTO', 'I-PROCEDIMIENTO']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,9220,"[En, la, exploración, pélvica, destaca, útero, miomatoso, .]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O]"
1,1476,"[La, exploración, física, no, reveló, hallazgos, de, interés, .]","[O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O]"
2,9423,"[Exfumador, desde, hacía, 10, años, de, 15, paquetes, /, año, y, tuberculosis, pulmonar, (, TBP, ), en, el, año, 2000, tratada, correctamente, con, recidiva, en, 2003, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,4980,"[La, exploración, biomicroscópica, y, tonométrica, fue, normal, .]","[O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O]"
4,6687,"[El, paciente, eliminó, una, gran, cantidad, de, heces, y, gases, ,, permaneciendo, asintomático, a, las, 24, -, 48, horas, ,, por, lo, que, recibió, el, alta, hospitalaria, ,, siendo, remitido, al, servicio, de, cirugía, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
5,2712,"[Presentaba, hipertensión, arterial, de, 2, años, de, evolución, en, tratamiento, con, amlodipino, ,, atenolol, e, irbesartán, desde, hacía, años, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,2947,"[Se, introdujo, progresivamente, nutrición, enteral, por, vía, oral, y, dieta, oral, modificada, y, se, suspendió, la, NPT, y, los, fluidos, i, ., v, ., Dado, que, el, diagnóstico, de, insuficiencia, suprarrenal, no, estaba, confirmado, y, existía, al, ingreso, una, importante, deshidratación, que, podría, explicar, las, alteraciones, analíticas, ,, se, realizó, un, descenso, progresivo, de, los, corticoides, hasta, suspenderlos, ,, sin, que, apareciesen, síntomas, o, signos, de, insuficiencia, suprarrenal, .]","[O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, B-PROCEDIMIENTO, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,10381,"[El, periodo, neonatal, cursó, sin, incidencias, ,, presentando, un, desarrollo, psicomotor, y, ponderoestatural, adecuado, hasta, los, dos, meses, de, edad, ,, cuando, comenzó, con, la, clínica, descrita, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,5608,"[La, evolución, postoperatoria, fue, satisfactoria, ,, con, edema, discreto, de, la, mano, que, cedió, al, poco, tiempo, ;, el, paciente, recuperó, la, movilidad, de, los, dedos, y, no, sufrió, alteración, de, la, sensibilidad, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,7255,"[Posteriormente, ,, se, procede, a, una, exploración, inicial, y, diagnóstica, ,, donde, se, evidencia, una, herida, incisocontusa, y, anfractuosa, desde, la, región, supraclavicular, izquierda, alta, hasta, la, comisura, labial, ipsilateral, .]","[O, O, O, O, O, O, B-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 185, 7409, 9080, 1162, 15661, 1320, 1260, 2588, 1126, 2155, 14255, 14601, 7409, 1988, 4035, 17404, 7298, 1260, 2774, 1162, 4167, 11252, 1186, 2885, 7971, 1260, 1299, 5970, 10243, 3840, 6922, 1162, 8362, 1161, 12365, 11708, 1643, 24171, 16012, 117, 182, 3329, 15435, 185, 1766, 8468, 16408, 1348, 170, 10182, 2007, 170, 27231, 1161, 1260, 190, 13166, 8032, 1465, 1274, 2007, 14516, 1842, 23228, 4252, 1643, 24171, 16012, 20497, 5053, 2599, 117, 170, 1643, 1874, 13017, 2572, 12477, 3202, 1260, 122, 1665, 1306, 170, 1643, 2180, 8745, 22678, 1186, 1260, 4267, 16470, 8005, 12864, 9080, 1162, 1260, 174, 25786, 2386, 12013, 117, 194, 174, 2528, 14867, 18771, 2774, 20899, 117, 15027, 14516, 12862, 1161, 3254, 1186, 8241, 1988, 6873, 5552, 4600, 1161, 4035, 10347, 6409, 1161, 1260, 174, 25786, 2386, 12013, 4167, 11252, 1186, 119, 102], [101, 14516, 1842, 23228, 187, 1306, 1179, 119, 102], [101, 12434, 5709, 1186, 12477, 3202, 6873, 5552, 117, 27466, 6696, 1186, 8468, 14601, 805

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/10709 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"/content/drive/MyDrive/CLEF2023/single-{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy = "epoch",
    push_to_hub=False,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-33-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PROCEDIMIENTO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.296000,0.185372,0.717668,0.785839,0.750208,0.931209
2,0.172000,0.172588,0.711392,0.814261,0.759358,0.936668
3,0.111800,0.179526,0.733333,0.811768,0.770560,0.938306


TrainOutput(global_step=2010, training_loss=0.1791717919544201, metrics={'train_runtime': 797.0885, 'train_samples_per_second': 40.305, 'train_steps_per_second': 2.522, 'total_flos': 2099317440519054.0, 'train_loss': 0.1791717919544201, 'epoch': 3.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.17952580749988556,
 'eval_precision': 0.7333333333333333,
 'eval_recall': 0.8117676389927699,
 'eval_f1': 0.7705596970772688,
 'eval_accuracy': 0.9383062692061709,
 'eval_runtime': 12.2999,
 'eval_samples_per_second': 96.748,
 'eval_steps_per_second': 6.098,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'PROCEDIMIENTO': {'precision': 0.7333333333333333,
  'recall': 0.8117676389927699,
  'f1': 0.7705596970772688,
  'number': 4011},
 'overall_precision': 0.7333333333333333,
 'overall_recall': 0.8117676389927699,
 'overall_f1': 0.7705596970772688,
 'overall_accuracy': 0.9383062692061709}